In [192]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo"

In [193]:
# 使用了retry库，指定在请求失败时的重试策略。
# 这里设定的是指数等待（wait_random_exponential），时间间隔的最大值为40秒，并且最多重试3次（stop_after_attempt(3)）。
# 定义一个函数chat_completion_request，主要用于发送 聊天补全 请求到OpenAI服务器
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    # 设定请求的header信息，包括 API_KEY
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
,
    }
    # 设定请求的JSON数据，包括GPT 模型名和要进行补全的消息
    json_data = {"model":model,"messages":messages}
    # 如果传入了functions，将其加入到json_data中
    if functions is not None:
        json_data.update({"functions": functions})

    # 如果传入了function_call，将其加入到json_data中
    if function_call is not None:
        json_data.update({"function_call": function_call})

    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        # 返回服务器的响应
        return response

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [194]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):
    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    # 遍历消息列表
    for message in messages:

        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [195]:
# 定义一个名为functions的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数  分别是获取当前天气和获取未来N天的天气预报。

# 第一个字典定义了一个名为"get_current_weather"的功能
functions = [
    {
        "name": "get_current_weather",  # 功能的名称
        "description": "Get the current weather",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该功能需要的必要参数
        },
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该功能需要的必要参数
        },
    },
]

In [196]:
# 这段代码首先定义了一个messages列表用来存储聊天的消息，然后向列表中添加了系统和用户的消息。

# 然后，它使用了之前定义的chat_completion_request函数发送一个请求，传入的参数包括消息列表和函数列表。

# 在接收到响应后，它从JSON响应中解析出助手的消息，并将其添加到消息列表中。

# 最后，它打印出 GPT 模型回复的消息。

# （如果我们询问当前天气，GPT 模型会回复让你给出更准确的问题。）

In [198]:
print(assistant_message)

def get_current_weather(location):
   url= "http://api.openweathermap.org/data/2.5/forecast?q=shanghai,CN&APPID=917c9f8e898e0ecbd6985e1744d93e9f"
   r = requests.get(url)  
   j = r.json()['list']    # 返回40条， j[0] 查看第一条数据       
         
   i=5
   while i < 40:
        print('\nDay', int((i-2)/8) )   
        print('预报时刻：' , j[i]['dt_txt'])
        print('温度    ：' , '%.2f'%(j[i]['main']['temp'] -273.15))    # 注意 '%.2f'% 的用法
        print('湿度    ：' , j[i]['main']['humidity'])
        print('气压    ：' , j[i]['main']['pressure'])
        print('天气    ：' , j[i]['weather'][0]['description'])
        print('风速    ：' , j[i]['wind']['speed'])
        print('能见度  ：' , j[i]['visibility'])
        i += 8


def execute_function_call(message):
    """执行函数调用"""
    # 判断功能调用的名称是否为 "ask_database"
    if message["function_call"]["name"] == "get_current_weather":
        # 如果是，则获取功能调用的参数，这里是 SQL 查询
        query = json.loads(message["function_call"]["arguments"])
        print(query)
        # 使用 ask_database 函数执行查询，并获取结果
        results = get_current_weather(query)
    else:
        # 如果功能调用的名称不是 "ask_database"，则返回错误信息
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results  # 返回结果

{'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Shanghai, China",\n  "format": "celsius"\n}'}}


In [199]:
print(chat_response)

<Response [429]>


In [205]:
# 向messages列表添加一条用户角色的消息，用户告知他们在苏格兰的格拉斯哥
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "I'm in Shanghai, China."  # 用户的消息内容
})

# 再次使用定义的chat_completion_request函数发起一个请求，传入更新后的messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的新的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的消息添加到消息列表中
messages.append(assistant_message)

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    #print(assistant_message["function_call"]["arguments"].replace('{','').replace('}',''))
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})

pretty_print_conversation(messages)

{'location': 'Shanghai, China', 'format': 'celsius'}

Day 0
预报时刻： 2023-07-30 03:00:00
温度    ： 28.81
湿度    ： 70
气压    ： 1007
天气    ： overcast clouds
风速    ： 7.4
能见度  ： 10000

Day 1
预报时刻： 2023-07-31 03:00:00
温度    ： 31.03
湿度    ： 63
气压    ： 1007
天气    ： scattered clouds
风速    ： 4.56
能见度  ： 10000

Day 2
预报时刻： 2023-08-01 03:00:00
温度    ： 29.94
湿度    ： 66
气压    ： 1002
天气    ： light rain
风速    ： 6.3
能见度  ： 10000

Day 3
预报时刻： 2023-08-02 03:00:00
温度    ： 27.35
湿度    ： 84
气压    ： 991
天气    ： light rain
风速    ： 10.91
能见度  ： 10000

Day 4
预报时刻： 2023-08-03 03:00:00
温度    ： 25.23
湿度    ： 95
气压    ： 991
天气    ： light rain
风速    ： 8.95
能见度  ： 10000
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

user: I'm in Shanghai, China.

user: I'm in Shanghai, China.

user: I'm in Shanghai, China.

user: I'm in Shanghai, China.

user: I'm in Shanghai, China.

user: I'm in Shanghai, China.

assistan